In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from SimulateDay import *

portfolio = pd.read_csv('portfolio.csv')

In [3]:
def execute_sell(fee, SalePrice, stock):
    if SalePrice == 0:
        return 0
    percentage = (fee / SalePrice) * 100
    portfolio = pd.read_csv('portfolio.csv')
    
    # Update the SalePrice for the given stock on the most recent day
    portfolio.loc[(portfolio['Stock Name'] == stock) & (portfolio['Day'] == portfolio['Day'].max()), 'SalePrice'] = SalePrice - fee
    
    # Write to the log file in append mode
    with open('Log.txt', 'a') as file:
        file.write(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}\n")
    
    # Print the log message
    print(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}")
    
    return portfolio


def execute_buy(stock_name, purchase_price):
    portfolio = pd.read_csv('portfolio.csv')
    stock = portfolio[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max())]
    
    if len(stock) == 0:
        # Fetch stock data
        stock_data = get_stock_data(stock_name)
        stock_price = stock_data['Close'].iloc[-1]
        
        # Calculate the number of shares that can be bought
        shares = purchase_price / stock_price
        
        # Append the new transaction to the portfolio
        portfolio = portfolio.append({
            'Stock Name': stock_name,
            'Day': portfolio['Day'].max() + 1,
            'Action': 'Buy',
            'Stock Price': stock_price,
            'Cash': purchase_price - (shares * stock_price),
            'Shares Held': shares,
            'Portfolio Value': shares * stock_price,
            'Date': stock_data['Date'].iloc[-1],
            'Actual Sell': 0
        }, ignore_index=True)
    else:
        stock_price = stock['Stock Price'].values[0]
        shares_to_buy = purchase_price / stock_price
        shares = stock['Shares Held'].values[0] + shares_to_buy
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Shares Held'] += shares_to_buy 
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Actual Buy'] = True
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Portfolio Value'] = (shares * stock_price)

    portfolio.to_csv('portfolio.csv', index=False)
    
    return portfolio

In [4]:
portfolio.tail(7)#['Portfolio Value'].sum()  

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date,Actual Sell,Actual Buy
98,AAPL,14,Buy,224.565002,0,0.331895,74.532029,2024-11-15 00:00:00-05:00,0.0,NaN
99,XOM,14,Buy,119.644997,0,0.896151,107.219978,2024-11-15 00:00:00-05:00,0.0,NaN
100,AMZN,14,Sell,200.960007,0,0.771653,155.071329,2024-11-15 00:00:00-05:00,0.0,NaN
101,LYV,14,Buy,128.585007,0,0.591747,76.089741,2024-11-15 00:00:00-05:00,0.0,NaN
102,INTC,14,Sell,24.410000,0,3.211000,78.380512,2024-11-15 00:00:00-05:00,0.0,NaN
103,DNMR,14,Hold,8.380000,0,1.820000,15.251600,2024-11-15 00:00:00-05:00,0.0,NaN
104,SPCE,14,Hold,6.820000,0,0.246548,1.681460,2024-11-15 00:00:00-05:00,0.0,NaN


In [5]:
fee = 0.02
SalePrice = 124.13
stock = ""
# Sale = execute_sell(fee, SalePrice, stock).fillna(0)

, sold at 124.11, fee: 0.02, percentage: 0.016112140497865142


In [6]:
purchase_price = 20
stock_name = "AAPL"
portfolio = execute_buy(stock_name, purchase_price)
portfolio.tail(7)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date,Actual Sell,Actual Buy
98,AAPL,14,Buy,224.565002,0,0.420956,94.532029,2024-11-15 00:00:00-05:00,0.0,True
99,XOM,14,Buy,119.644997,0,0.896151,107.219978,2024-11-15 00:00:00-05:00,0.0,NaN
100,AMZN,14,Sell,200.960007,0,0.771653,155.071329,2024-11-15 00:00:00-05:00,0.0,NaN
101,LYV,14,Buy,128.585007,0,0.591747,76.089741,2024-11-15 00:00:00-05:00,0.0,NaN
102,INTC,14,Sell,24.410000,0,3.211000,78.380512,2024-11-15 00:00:00-05:00,0.0,NaN
103,DNMR,14,Hold,8.380000,0,1.820000,15.251600,2024-11-15 00:00:00-05:00,0.0,NaN
104,SPCE,14,Hold,6.820000,0,0.246548,1.681460,2024-11-15 00:00:00-05:00,0.0,NaN


In [7]:
20/22.205000

0.900698040981761